# Fine-tuning llama2 (Youri-7b)

* https://www.kaggle.com/code/philculliton/fine-tuning-with-llama-2-qlora
* https://www.kaggle.com/code/hhoang41/llama-2-fine-tuning-using-qlora
* https://colab.research.google.com/drive/1tG9eqttfnqHoQqmsiacywUG9ilUhoiCk?usp=sharing#scrollTo=5o3OgCMdRGgp

In [1]:
# ! pip install accelerate peft bitsandbytes transformers trl

In [1]:
import os
import torch
from datasets import load_from_disk
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
import pandas as pd

/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


In [2]:
dataset = load_from_disk("converted")
dataset = dataset.map(lambda example: {"text": "ニュース:" + example["topic"] + "\n5chの反応:" + example["comment"]}, remove_columns=["topic", "comment"])
train_dataset = dataset['train']
eval_dataset = dataset['test']

train_dataset = train_dataset.select(range(int(len(train_dataset) * 0.01)))
eval_dataset = eval_dataset.select(range(int(len(eval_dataset) * 0.01)))

In [4]:
compute_dtype = getattr(torch, "float16")

quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=False,
)

In [5]:
model = AutoModelForCausalLM.from_pretrained(
    "rinna/youri-7b",
    quantization_config=quant_config,
    device_map={"": 0}
)
model.config.use_cache = False
model.config.pretraining_tp = 1

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [6]:
tokenizer = AutoTokenizer.from_pretrained("rinna/youri-7b", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

In [7]:
peft_args = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [8]:
training_params = TrainingArguments(
    output_dir="./results",
    num_train_epochs=1,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    optim="paged_adamw_32bit",
    save_steps=25,
    logging_steps=25,
    eval_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=False,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard"
)

In [9]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_args,
    dataset_text_field="text",
    max_seq_length=None,
    tokenizer=tokenizer,
    args=training_params,
    packing=False,
)

/home/peryaudo/mambaforge/envs/peryagpt/lib/python3.11/site-packages/trl/trainer/sft_trainer.py:194: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/191 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

In [ ]:
# TODO: >>1 tends to be too long for the context length. Is it possible to truncate it beforehand?
# TODO: Check if the fine tuned weight has better performance than youri-7b-instruction
# TODO: Pretrain LLM with 5ch style posts beforehand (like ULMFiT)